In [1]:
import re
import pandas as pd

# The extracted text from OCR
extracted_text = '''
Following the claim under Terminal illness benefit, if the Paid up sum assured is higher than the terminal benefit paid, the balance death benefit, will be paid on death of the life insured during the policy term.

The Paid-up Sum Assured on death = {Sum Assured on death *(No. of premiums paid/Total no. of premiums payable)}.

After payment of Terminal illness benefit, the Paid up sum assured would be constant and no further increase of sum assured will be allowed under the plan option: Increasing Cover.

If Better Half Benefit is opted, then on death of the Life assured during the policy term, provided the policy is in Reduced paid up status, the spouse would be covered for the Better Half Paid up Sum assured during the spouse coverage term, Where Better Half Paid-up sum assured = Better Half sum assured *No. of premiums paid / No. of premiums originally payable.

Surrender benefit
For Regular Premium: No Surrender benefit is available.

For Single Premium: The policy can be surrendered anytime during the policy term. The surrender value payable is: 70% of Single Premium *(Unexpired Policy Term/Original Policy Term).

For Limited Premium payment: The policy acquires Surrender value only if at least first 2 full consecutive policy year’s premiums have been paid. The surrender value payable is: Surrender value factor *(Unexpired Policy Term/Original Policy Term) * Total premiums paid.

Surrender Value Factors
Policy Year                Surrender Value Factor
1                          0%
2                          30%
3                          40%
4 to the end of Premium Payment Term 50%
For the rest of the policy term 70%

Single Premium / Total Premiums paid / received means total of all the premiums received, excluding any extra premium, any rider premium and applicable taxes.

Sample Premium Rates
The table below shows indicative annual premiums excluding applicable taxes for a Life cover of ₹1 crore.
'''

# Extract the table text using regex
table_text = re.search(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", extracted_text, re.DOTALL).group(1)

# Split the table text into lines
lines = table_text.strip().split('\n')

# Extract the table rows
rows = []
for line in lines:
    # Split the line into columns by whitespace
    columns = re.split(r'\s{2,}', line.strip())
    if len(columns) == 2:
        rows.append(columns)
    elif len(columns) > 2:  # handle case for "4 to the end of Premium Payment Term"
        rows.append([' '.join(columns[:-1]), columns[-1]])

# Create a DataFrame
df = pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"])
df


,Policy Year,Surrender Value Factor
0,1,0%
1,2,30%
2,3,40%


In [17]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [22]:
import os
from PIL import Image
import pytesseract
import re
import pandas as pd

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images

def extract_text_from_images(images):
    extracted_texts = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_texts.append(text)
    return extracted_texts

def save_texts_to_files(extracted_texts, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for i, text in enumerate(extracted_texts):
        with open(os.path.join(output_folder, f'extracted_text_{i+1}.txt'), 'w', encoding='utf-8') as file:
            file.write(text)

def search_keywords_in_texts(keywords, extracted_texts):
    keyword_texts = []
    for text in extracted_texts:
        for keyword in keywords:
            if keyword in text:
                keyword_texts.append(text)
                break  # Avoid adding the same text multiple times
    return keyword_texts

# Extract and parse table from text
def extract_tables_from_text(text):
    tables = []
    matches = re.findall(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", text, re.DOTALL)
    for match in matches:
        lines = match.strip().split('\n')
        rows = []
        for line in lines:
            columns = re.split(r'\s{2,}', line.strip())
            if len(columns) == 2:
                rows.append(columns)
            elif len(columns) > 2:  # handle case for "4 to the end of Premium Payment Term"
                rows.append([' '.join(columns[:-1]), columns[-1]])
        tables.append(pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"]))
    return tables

def save_tables_to_csv(tables, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for i, table in enumerate(tables):
        table.to_csv(os.path.join(output_folder, f'table_{i+1}.csv'), index=False)

# Main function to execute the steps
def main(image_folder, output_text_folder, output_keyword_folder, output_table_folder, keywords):
    images = load_images_from_folder(image_folder)
    extracted_texts = extract_text_from_images(images)
    save_texts_to_files(extracted_texts, output_text_folder)
    
    keyword_texts = search_keywords_in_texts(keywords, extracted_texts)
    save_texts_to_files(keyword_texts, output_keyword_folder)
    
    for text in keyword_texts:
        tables = extract_tables_from_text(text)
        save_tables_to_csv(tables, output_table_folder)

# Example usage
image_folder_path = r'E:\text_ext\img'
output_text_folder_path = r'E:\text_ext\layout'
output_keyword_folder_path = r'E:\text_ext\op1'
output_table_folder_path = r'E:\text_ext\op2'
keywords = ['Surrender benefit', 'Paid-up Sum Assured', 'Terminal illness benefit']

main(image_folder_path, output_text_folder_path, output_keyword_folder_path, output_table_folder_path, keywords)


In [9]:
import os
from PIL import Image
import pytesseract
import re
import pandas as pd

# Specify the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
                filenames.append(filename)
    return images, filenames

def extract_text_from_images(images):
    extracted_texts = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_texts.append(text)
    return extracted_texts

def save_texts_to_files(extracted_texts, filenames, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for text, filename in zip(extracted_texts, filenames):
        base_filename = os.path.splitext(filename)[0]  # Get the base filename without extension
        with open(os.path.join(output_folder, f'{base_filename}.txt'), 'w', encoding='utf-8') as file:
            file.write(text)

def search_keywords_in_texts(keywords, extracted_texts):
    keyword_texts = []
    for text in extracted_texts:
        for keyword in keywords:
            if keyword in text:
                keyword_texts.append(text)
                break  # Avoid adding the same text multiple times
    return keyword_texts

# Extract and parse table from text
def extract_tables_from_text(text):
    tables = []
    matches = re.findall(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", text, re.DOTALL)
    for match in matches:
        lines = match.strip().split('\n')
        rows = []
        for line in lines:
            columns = re.split(r'\s{2,}', line.strip())
            if len(columns) == 2:
                rows.append(columns)
            elif len(columns) > 2:  # handle case for "4 to the end of Premium Payment Term"
                rows.append([' '.join(columns[:-1]), columns[-1]])
        tables.append(pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"]))
    return tables

def save_tables_to_csv(tables, filenames, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for i, (table, filename) in enumerate(zip(tables, filenames)):
        base_filename = os.path.splitext(filename)[0]  # Get the base filename without extension
        table.to_csv(os.path.join(output_folder, f'{base_filename}_table.csv'), index=False)

# Main function to execute the steps
def main(image_folder, output_text_folder, output_keyword_folder, output_table_folder, keywords):
    images, filenames = load_images_from_folder(image_folder)
    extracted_texts = extract_text_from_images(images)
    save_texts_to_files(extracted_texts, filenames, output_text_folder)
    
    keyword_texts = search_keywords_in_texts(keywords, extracted_texts)
    save_texts_to_files(keyword_texts, filenames, output_keyword_folder)
    
    all_tables = []
    for text in keyword_texts:
        tables = extract_tables_from_text(text)
        all_tables.extend(tables)
    save_tables_to_csv(all_tables, filenames, output_table_folder)

image_folder_path = r'E:\text_ext\image1'
output_text_folder_path = r'E:\text_ext\layout'
output_keyword_folder_path = r'E:\text_ext\op1'
output_table_folder_path = r'E:\text_ext\op2'
keywords = ['Surrender benefit', 'Paid-up Sum Assured', 'Terminal illness benefit']

main(image_folder_path, output_text_folder_path, output_keyword_folder_path, output_table_folder_path, keywords)


In [11]:
import os
from PIL import Image
import pytesseract
import re
import pandas as pd

# Specify the path to the Tesseract executable
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_text_from_image(image_path):
    """Extract text from a single image."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_tables_from_text(text):
    """Extract tables from the text using regular expressions."""
    tables = []
    matches = re.findall(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", text, re.DOTALL)
    for match in matches:
        lines = match.strip().split('\n')
        rows = []
        for line in lines:
            columns = re.split(r'\s{2,}', line.strip())
            if len(columns) == 2:
                rows.append(columns)
            elif len(columns) > 2:  # handle case for "4 to the end of Premium Payment Term"
                rows.append([' '.join(columns[:-1]), columns[-1]])
        tables.append(pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"]))
    return tables

def save_extracted_content(image_path, output_text_path, output_table_folder):
    """Extract text and tables from an image and save them."""
    text = extract_text_from_image(image_path)
    
    # Save the extracted text
    with open(output_text_path, 'w', encoding='utf-8') as file:
        file.write(text)
    
    # Extract tables from text
    tables = extract_tables_from_text(text)
    
    # Save each table as a CSV file
    base_filename = os.path.splitext(os.path.basename(image_path))[0]
    if not os.path.exists(output_table_folder):
        os.makedirs(output_table_folder)
    for i, table in enumerate(tables):
        table.to_csv(os.path.join(output_table_folder, f'{base_filename}_table_{i+1}.csv'), index=False)

# Example usage
image_path = r'E:\text_ext\image1\SBI_life_page-0014.jpg'
output_text_path = r'E:\text_ext\layout\example.txt'
output_table_folder = r'E:\text_ext\op2'

save_extracted_content(image_path, output_text_path, output_table_folder)


In [17]:
import os
import pdfplumber
import camelot
import pandas as pd
import re

def clean_text(text):
    """Clean text by removing unwanted characters and placeholders."""
    text = re.sub(r'\(cid:\d+\)', '', text)  # Remove (cid:xxx) placeholders
    text = text.replace('\n', ' ').replace('\r', ' ')
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize whitespace
    return text

def extract_text_from_pdf(pdf_path):
    """Extract text from each page of the PDF using pdfplumber."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return clean_text(text)

def extract_tables_from_pdf(pdf_path):
    """Extract tables from a PDF using camelot."""
    tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
    return tables

def save_text_to_file(text, output_text_path):
    """Save the extracted text to a file."""
    with open(output_text_path, 'w', encoding='utf-8') as file:
        file.write(text)

def save_tables_to_csv(tables, output_table_folder, base_filename):
    """Save extracted tables to CSV files."""
    if not os.path.exists(output_table_folder):
        os.makedirs(output_table_folder)
    for i, table in enumerate(tables):
        table.to_csv(os.path.join(output_table_folder, f'{base_filename}_table_{i+1}.csv'), index=False)

# Example usage
pdf_path = r'E:\text_ext\SBI_life.pdf'
output_text_path = r'E:\text_ext\layout\extracted_text.txt'
output_table_folder = r'E:\text_ext\op2'

# Extract and save text from PDF
extracted_text = extract_text_from_pdf(pdf_path)
save_text_to_file(extracted_text, output_text_path)

# Extract and save tables from PDF
tables = extract_tables_from_pdf(pdf_path)
base_filename = os.path.splitext(os.path.basename(pdf_path))[0]
save_tables_to_csv(tables, output_table_folder, base_filename)


C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\camelot\parsers\stream.py:365: UserWarning: No tables found in table area 1
  warnings.warn(f"No tables found in table area {table_idx + 1}")


In [4]:
import cv2
# import pytesseract
import pandas as pd

# Function to extract text from an image
def extract_text(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    # Convert the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Use Tesseract to extract text
    text = pytesseract.image_to_string(gray)
    return text

# Function to extract tables from an image
def extract_tables(image_path):
    # Read the image using OpenCV
    image = cv2.imread(image_path)
    # Convert the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Use Tesseract to extract data
    data = pytesseract.image_to_data(gray, output_type=pytesseract.Output.DICT)
    
    # Extract text by each block and filter table-like structures
    tables = []
    n_boxes = len(data['text'])
    for i in range(n_boxes):
        if int(data['conf'][i]) > 60:  # Confidence level
            (x, y, w, h) = (data['left'][i], data['top'][i], data['width'][i], data['height'][i])
            tables.append(data['text'][i])

    # Convert list of table texts to a dataframe
    table_df = pd.DataFrame([x.split() for x in tables])
    return table_df

# Example usage
image_path = r'E:\text_ext\image1\SBI_life_page-0014.jpg'
text = extract_text(image_path)
print("Extracted Text:")
print(text)

table_df = extract_tables(image_path)
print("\nExtracted Table:")
print(table_df)


Extracted Text:
Whole Life:

Premium Payment Term

Other than Whole life

(in years)

LPPT (Years):
7, 10, 15, 20 and
25 years

For Single Premium:

One time at Policy inception

For Regular Premium:

Same as Policy Term

For Limited Premium: 5, 7, 10, 15, 20, 25 years and
(Policy Term less 5 years)

*Premium Amount Premium Frequency

Single

Yearly

Half Yearly

Monthly”

Minimum Premium | Maximum:

19,000 No Limit, subject to board
approved underwriting policy

3,600

1,836

306

All the references to age are age ason lastbirthday.

“Premium shown above are excluding applicable taxes and underwriting extra. Taxes will beapplicable as per the prevalent taxlaws
For Monthly mode, upto 3 Months premium to be paid in advance and renewal premium payment through Electronic Clearing System
(ECS) or Standing Instructions (where payment is made either by direct debit of bank account or credit card). For Monthly Salary Saving
Scheme (SSS) upto 2months premium tobe paid in advance and renewal pr

In [7]:
# import pytesseract
from PIL import Image
import os
import cv2
import camelot

# Set the path to the Tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'path_to_tesseract_executable'  # Replace with the actual path

# Folder containing the images
image_folder = r'E:\text_ext\image1'

# Output folder for text and tables
output_folder = r'E:\text_ext\op1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Extract text and save as .txt
for image_file in os.listdir(image_folder):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
        img_path = os.path.join(image_folder, image_file)
        img = Image.open(img_path)
        text = pytesseract.image_to_string(img)
        
        txt_output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.txt')
        with open(txt_output_path, 'w') as f:
            f.write(text)
        
        print(f'Text extracted and saved to {txt_output_path}')

# Assuming tables are in PDFs
pdf_folder = r'E:\text_ext'

# Extract tables and save as .csv
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.lower().endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        tables = camelot.read_pdf(pdf_path, pages='all')
        
        for i, table in enumerate(tables):
            csv_output_path = os.path.join(output_folder, os.path.splitext(pdf_file)[0] + f'_table_{i}.csv')
            table.to_csv(csv_output_path)
            
            print(f'Table extracted and saved to {csv_output_path}')


Text extracted and saved to E:\text_ext\op1\SBI_life_page-0011.txt
Text extracted and saved to E:\text_ext\op1\SBI_life_page-0012.txt
Text extracted and saved to E:\text_ext\op1\SBI_life_page-0014.txt


OSError: Ghostscript is not installed. You can install it using the instructions here: https://camelot-py.readthedocs.io/en/master/user/install-deps.html

In [8]:
!pip install tabula-py

Defaulting to user installation because normal site-packages is not writeable


In [10]:
import tabula

# Extract tables and save as .csv
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.lower().endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        tables = tabula.read_pdf(pdf_path, pages='all', multiple_tables=True)
        
        for i, table in enumerate(tables):
            csv_output_path = os.path.join(output_folder, os.path.splitext(pdf_file)[0] + f'_table_{i}.csv')
            table.to_csv(csv_output_path, index=False)
            
            print(f'Table extracted and saved to {csv_output_path}')

Failed to import jpype dependencies. Fallback to subprocess.
No module named 'jpype'


JavaNotFoundError: `java` command is not found from this Python process.Please ensure Java is installed and PATH is set for `java`

In [14]:
# import pytesseract
# from PIL import Image
# import os
# import cv2
# import pandas as pd

# # Set the path to the Tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'path_to_tesseract_executable'  # Replace with the actual path

# Folder containing the images
image_folder = r'E:\text_ext\image1'

# Output folder for text and tables
output_folder = r'E:\text_ext\op1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to extract text from images
def extract_text_from_image(image_path, output_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    with open(output_path, 'w') as f:
        f.write(text)

# Function to detect and extract tables from images
def extract_table_from_image(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    img_bin = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    img_bin = 255 - img_bin

    # Detecting horizontal and vertical lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))

    horizontal_lines = cv2.erode(img_bin, horizontal_kernel, iterations=2)
    horizontal_lines = cv2.dilate(horizontal_lines, horizontal_kernel, iterations=2)

    vertical_lines = cv2.erode(img_bin, vertical_kernel, iterations=2)
    vertical_lines = cv2.dilate(vertical_lines, vertical_kernel, iterations=2)

    # Combine horizontal and vertical lines
    table_mask = horizontal_lines + vertical_lines

    # Find contours for table cells
    contours, _ = cv2.findContours(table_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    table_data = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 50 and h > 20:  # Filter out small contours that are not likely to be table cells
            cell_img = img[y:y+h, x:x+w]
            cell_text = pytesseract.image_to_string(cell_img, config='--psm 6')
            table_data.append((x, y, cell_text.strip()))

    if table_data:
        # Sort by y coordinate, then by x coordinate
        table_data.sort(key=lambda x: (x[1], x[0]))

        # Group by rows
        rows = {}
        for x, y, text in table_data:
            if y not in rows:
                rows[y] = []
            rows[y].append(text)

        sorted_rows = [rows[key] for key in sorted(rows.keys())]

        # Save table data as CSV
        csv_output_path = os.path.join(output_path, os.path.splitext(os.path.basename(image_path))[0] + '.csv')
        df = pd.DataFrame(sorted_rows)
        df.to_csv(csv_output_path, index=False, header=False)
        print(f'Table extracted and saved to {csv_output_path}')

# Process images in the folder
for image_file in os.listdir(image_folder):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
        img_path = os.path.join(image_folder, image_file)
        
        # Extract text
        txt_output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.txt')
        extract_text_from_image(img_path, txt_output_path)
        print(f'Text extracted and saved to {txt_output_path}')
        
        # Extract table
        extract_table_from_image(img_path, output_folder)


Text extracted and saved to E:\text_ext\op1\SBI_life_page-0011.txt
Table extracted and saved to E:\text_ext\op1\SBI_life_page-0011.csv
Text extracted and saved to E:\text_ext\op1\SBI_life_page-0012.txt
Table extracted and saved to E:\text_ext\op1\SBI_life_page-0012.csv
Text extracted and saved to E:\text_ext\op1\SBI_life_page-0014.txt
Table extracted and saved to E:\text_ext\op1\SBI_life_page-0014.csv


In [12]:
import numpy as np
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [19]:

import pdfplumber

# Set the path to the Tesseract executable
# pytesseract.pytesseract.tesseract_cmd = r'path_to_tesseract_executable'  # Replace with the actual path

# Folder containing the images
image_folder =  r'E:\text_ext\image1'

# Folder containing the PDFs
pdf_folder =  r'E:\text_ext'

# Output folder for text and tables
output_folder =  r'E:\text_ext\op1'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Function to extract text from images
def extract_text_from_image(image_path, output_path):
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    with open(output_path, 'w') as f:
        f.write(text)

# Function to extract tables from PDFs
def extract_tables_from_pdf(pdf_path, output_folder):
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            tables = page.extract_tables()
            for j, table in enumerate(tables):
                if table:
                    df = pd.DataFrame(table[1:], columns=table[0])
                    csv_output_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(pdf_path))[0]}_page_{i+1}_table_{j+1}.csv")
                    df.to_csv(csv_output_path, index=False, encoding='utf-8')
                    print(f"Table extracted and saved to {csv_output_path}")

# Process images in the folder
for image_file in os.listdir(image_folder):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
        img_path = os.path.join(image_folder, image_file)
        
        # Extract text
        txt_output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.txt')
        extract_text_from_image(img_path, txt_output_path)
        print(f"Text extracted and saved to {txt_output_path}")

# Process PDFs in the folder
for pdf_file in os.listdir(pdf_folder):
    if pdf_file.lower().endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder, pdf_file)
        
        # Extract tables
        extract_tables_from_pdf(pdf_path, output_folder)

Text extracted and saved to E:\text_ext\op1\SBI_life_page-0011.txt
Text extracted and saved to E:\text_ext\op1\SBI_life_page-0012.txt
Text extracted and saved to E:\text_ext\op1\SBI_life_page-0014.txt
Table extracted and saved to E:\text_ext\op1\SBI_life_page_4_table_1.csv
Table extracted and saved to E:\text_ext\op1\SBI_life_page_6_table_1.csv
Table extracted and saved to E:\text_ext\op1\SBI_life_page_10_table_1.csv
Table extracted and saved to E:\text_ext\op1\SBI_life_page_11_table_1.csv
Table extracted and saved to E:\text_ext\op1\SBI_life_page_11_table_2.csv
Table extracted and saved to E:\text_ext\op1\SBI_life_page_11_table_3.csv


PermissionError: [Errno 13] Permission denied: 'E:\\text_ext\\op1\\SBI_life_page_13_table_1.csv'

In [21]:
from pandas_table_extractor import TableExtractor

ModuleNotFoundError: No module named 'pandas_table_extractor'

In [ ]:
# Function to extract tables from images
def extract_table_from_image(image_path, output_path):
    img = cv2.imread(image_path)
    te = TableExtractor(img)
    tables = te.extract_tables()
    if tables:
        for idx, table in enumerate(tables):
            df = pd.DataFrame(table)
            csv_output_path = os.path.join(output_path, f"{os.path.splitext(os.path.basename(image_path))[0]}_table_{idx + 1}.csv")
            df.to_csv(csv_output_path, index=False, encoding='utf-8')
            print(f"Table extracted and saved to {csv_output_path}")

# Process images in the folder
for image_file in os.listdir(image_folder):
    if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp')):
        img_path = os.path.join(image_folder, image_file)
        
        # Extract text
        txt_output_path = os.path.join(output_folder, os.path.splitext(image_file)[0] + '.txt')
        extract_text_from_image(img_path, txt_output_path)
        print(f"Text extracted and saved to {txt_output_path}")
        
        # Extract table
        extract_table_from_image(img_path, output_folder)

# Test with uploaded image
uploaded_image_path = '/mnt/data/image.png'
extract_text_from_image(uploaded_image_path, '/mnt/data/uploaded_image_text.txt')
extract_table_from_image(uploaded_image_path, '/mnt/data')
Explanation:
Language Setting for OCR: Set the lang parameter in pytesseract.image_to_string to 'eng' to specify English language recognition, which helps improve accuracy.
Encoding: Ensure the output files are saved with utf-8 encoding to handle non-ASCII characters correctly.
Table Extraction: Uses pandas-table-extractor for robust table detection and extraction from images.
Testing with Uploaded Image
To verify, we'll run the text and table extraction on the uploaded image:

python
Copy code
# Extract text from the uploaded image
extract_text_from_image(uploaded_image_path, '/mnt/data/uploaded_image_text.txt')

# Extract tables from the uploaded image
extract_table_from_image(uploaded_image_path, '/mnt/data')

In [ ]:
import os
from PIL import Image
import pytesseract
import re
import pandas as pd

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images

def extract_text_from_images(images):
    extracted_texts = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_texts.append(text)
    return extracted_texts

def search_keywords_in_texts(keywords, extracted_texts):
    keyword_info = []
    for text in extracted_texts:
        for keyword in keywords:
            if keyword in text:
                keyword_info.append((keyword, text))
                break  # Avoid adding the same text multiple times
    return keyword_info

def extract_tables_from_text(text):
    tables = []
    matches = re.findall(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", text, re.DOTALL)
    for match in matches:
        lines = match.strip().split('\n')
        rows = []
        for line in lines:
            columns = re.split(r'\s{2,}', line.strip())
            if len(columns) == 2:
                rows.append(columns)
            elif len(columns) > 2:  # Handle case for "4 to the end of Premium Payment Term"
                rows.append([' '.join(columns[:-1]), columns[-1]])
        tables.append(pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"]))
    return tables

def search_keywords_in_tables(keywords, tables):
    keyword_info = []
    for table in tables:
        for keyword in keywords:
            if table.to_string().find(keyword) != -1:
                keyword_info.append((keyword, table))
                break  # Avoid adding the same table multiple times
    return keyword_info

def save_keyword_info_to_dataframe(keyword_info, output_file):
    data = []
    for keyword, info in keyword_info:
        if isinstance(info, pd.DataFrame):
            info_str = info.to_string(index=False)
        else:
            info_str = info
        data.append({"Keyword": keyword, "Information": info_str})
    
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)

def main(image_folder, output_file, keywords):
    images = load_images_from_folder(image_folder)
    extracted_texts = extract_text_from_images(images)
    
    text_keyword_info = search_keywords_in_texts(keywords, extracted_texts)
    all_tables = []
    for _, text in text_keyword_info:
        all_tables.extend(extract_tables_from_text(text))
    
    table_keyword_info = search_keywords_in_tables(keywords, all_tables)
    combined_keyword_info = text_keyword_info + table_keyword_info
    
    save_keyword_info_to_dataframe(combined_keyword_info, output_file)

# Example usage
image_folder_path = r'E:\text_ext\img'
output_file_path = r'E:\text_ext\keyword_info.csv'
keywords = ['Surrender benefit', 'Paid-up Sum Assured', 'Terminal illness benefit']

main(image_folder_path, output_file_path, keywords)


In [23]:
import os
from PIL import Image
import pytesseract
import re
import pandas as pd

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images

def extract_text_from_images(images):
    extracted_texts = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_texts.append(text)
    return extracted_texts

def save_all_text_to_file(extracted_texts, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for text in extracted_texts:
            file.write(text + "\n\n")

def search_keywords_in_texts(keywords, extracted_texts):
    keyword_info = []
    for text in extracted_texts:
        for keyword in keywords:
            if keyword in text:
                keyword_info.append((keyword, text))
                break  # Avoid adding the same text multiple times
    return keyword_info

def extract_tables_from_text(text):
    tables = []
    matches = re.findall(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", text, re.DOTALL)
    for match in matches:
        lines = match.strip().split('\n')
        rows = []
        for line in lines:
            columns = re.split(r'\s{2,}', line.strip())
            if len(columns) == 2:
                rows.append(columns)
            elif len(columns) > 2:  # Handle case for "4 to the end of Premium Payment Term"
                rows.append([' '.join(columns[:-1]), columns[-1]])
        tables.append(pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"]))
    return tables

def search_keywords_in_tables(keywords, tables):
    keyword_info = []
    for table in tables:
        for keyword in keywords:
            if table.to_string().find(keyword) != -1:
                keyword_info.append((keyword, table))
                break  # Avoid adding the same table multiple times
    return keyword_info

def save_keyword_info_to_dataframe(keyword_info, output_file):
    data = []
    for keyword, info in keyword_info:
        if isinstance(info, pd.DataFrame):
            info_str = info.to_string(index=False)
        else:
            info_str = info
        data.append({"Keyword": keyword, "Information": info_str})
    
    df = pd.DataFrame(data)
    df.to_csv(output_file, index=False)

def main(image_folder, output_text_file, output_keyword_file, keywords):
    images = load_images_from_folder(image_folder)
    extracted_texts = extract_text_from_images(images)
    
    save_all_text_to_file(extracted_texts, output_text_file)
    
    text_keyword_info = search_keywords_in_texts(keywords, extracted_texts)
    all_tables = []
    for _, text in text_keyword_info:
        all_tables.extend(extract_tables_from_text(text))
    
    table_keyword_info = search_keywords_in_tables(keywords, all_tables)
    combined_keyword_info = text_keyword_info + table_keyword_info
    
    save_keyword_info_to_dataframe(combined_keyword_info, output_keyword_file)

# Example usage
image_folder_path = r'E:\text_ext\image1'
output_text_file_path = r'E:\text_ext\OP1\all_texts.txt'
output_keyword_file_path = r'E:\text_ext\op1\keyword_info.csv'
keywords = ['Surrender benefit', 'Paid-up Sum Assured', 'Terminal illness benefit']

main(image_folder_path, output_text_file_path, output_keyword_file_path, keywords)


In [25]:
# Here's the updated script to create a DataFrame that shows the keywords and the
# related text information directly, without saving them separately first:

def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = Image.open(os.path.join(folder, filename))
            if img is not None:
                images.append(img)
    return images

def extract_text_from_images(images):
    extracted_texts = []
    for image in images:
        text = pytesseract.image_to_string(image)
        extracted_texts.append(text)
    return extracted_texts

def save_all_text_to_file(extracted_texts, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        for text in extracted_texts:
            file.write(text + "\n\n")

def search_keywords_in_texts(keywords, extracted_texts):
    keyword_info = []
    for text in extracted_texts:
        for keyword in keywords:
            if keyword in text:
                keyword_info.append((keyword, text))
                break  # Avoid adding the same text multiple times
    return keyword_info

def extract_tables_from_text(text):
    tables = []
    matches = re.findall(r"Surrender Value Factors\s*Policy Year\s*Surrender Value Factor\s*(.*?)\s*Single Premium", text, re.DOTALL)
    for match in matches:
        lines = match.strip().split('\n')
        rows = []
        for line in lines:
            columns = re.split(r'\s{2,}', line.strip())
            if len(columns) == 2:
                rows.append(columns)
            elif len(columns) > 2:  # Handle case for "4 to the end of Premium Payment Term"
                rows.append([' '.join(columns[:-1]), columns[-1]])
        tables.append(pd.DataFrame(rows, columns=["Policy Year", "Surrender Value Factor"]))
    return tables

def search_keywords_in_tables(keywords, tables):
    keyword_info = []
    for table in tables:
        for keyword in keywords:
            if table.to_string().find(keyword) != -1:
                keyword_info.append((keyword, table))
                break  # Avoid adding the same table multiple times
    return keyword_info

def save_keyword_info_to_dataframe(keyword_info, output_file):
    data = []
    for keyword, info in keyword_info:
        if isinstance(info, pd.DataFrame):
            info_str = info.to_string(index=False)
        else:
            info_str = info


In [26]:
import pandas as pd
import re

# Sample text data
text = """
Order ID: INF123q34gt04q
Purchase Type: Online
Amount: $1500
Value Date: 2024-07-28

Order ID: INF456y78hj90u
Purchase Type: In-Store
Amount: $250
Value Date: 2024-08-05
"""

# Define regex patterns for each keyword
patterns = {
    "Order ID": r"Order ID:\s*(INF\w+)",
    "Purchase Type": r"Purchase Type:\s*([a-zA-Z\s\-]+)",
    "Amount": r"Amount:\s*\$([0-9,\.]+)",
    "Value Date": r"Value Date:\s*([0-9\-]+)"
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    for match in re.finditer(r'Order ID:.*?(?=Order ID:|$)', text, re.DOTALL):
        order_text = match.group()
        data = {}
        for key, pattern in patterns.items():
            result = re.search(pattern, order_text)
            data[key] = result.group(1) if result else None
        extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


         Order ID     Purchase Type Amount  Value Date
0  INF123q34gt04q    Online\nAmount   1500  2024-07-28
1  INF456y78hj90u  In-Store\nAmount    250  2024-08-05


In [27]:
import pandas as pd
import re

# Sample text data
text = """
Folio number = 15380577 / 05
Company name = madame ltd.
ISIN number = INF456y78hj90u
Value date = 11/02/2021
Transaction type = Purchase
Amount in INR = 1500
NAV in INR = 200
Price in INR = 100
Number of units = 15

Folio number = 15489856 / 75
Company name = madame ltd.
ISIN number = INF123q34gt04q
Value date = 11/03/2021
Transaction type = Redemption
Amount in INR = 2500
NAV in INR = 300
Price in INR = 150
Number of units = 20
"""

# Define regex patterns for each keyword
patterns = {
    "Folio number": r"Folio number\s*=\s*([\d\s/]+)",
    "Company name": r"Company name\s*=\s*([\w\s.]+)",
    "ISIN number": r"ISIN number\s*=\s*(INF\w+)",
    "Value date": r"Value date\s*=\s*([\d/]+)",
    "Transaction type": r"Transaction type\s*=\s*([\w\s]+)",
    "Amount in INR": r"Amount in INR\s*=\s*([\d,\.]+)",
    "NAV in INR": r"NAV in INR\s*=\s*([\d,\.]+)",
    "Price in INR": r"Price in INR\s*=\s*([\d,\.]+)",
    "Number of units": r"Number of units\s*=\s*([\d,\.]+)"
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    for match in re.finditer(r'Folio number.*?(?=Folio number|$)', text, re.DOTALL):
        block_text = match.group()
        data = {}
        for key, pattern in patterns.items():
            result = re.search(pattern, block_text)
            data[key] = result.group(1).strip() if result else None
        extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


    Folio number              Company name     ISIN number  Value date  \
0  15380577 / 05  madame ltd.\nISIN number  INF456y78hj90u  11/02/2021   
1  15489856 / 75  madame ltd.\nISIN number  INF123q34gt04q  11/03/2021   

            Transaction type Amount in INR NAV in INR Price in INR  \
0    Purchase\nAmount in INR          1500        200          100   
1  Redemption\nAmount in INR          2500        300          150   

  Number of units  
0              15  
1              20  


In [28]:
df

,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,madame ltd.\nISIN number,INF456y78hj90u,11/02/2021,Purchase\nAmount in INR,1500,200,100,15
1,15489856 / 75,madame ltd.\nISIN number,INF123q34gt04q,11/03/2021,Redemption\nAmount in INR,2500,300,150,20


In [33]:
import pandas as pd
import re

# Sample text data
text = """
Order ID: INF123q34gt04q with Folio number 15380577 / 05 bought some stocks from madame ltd. 
ISIN number INF456y78hj90u on Value date 11/02/2021. The Transaction type was Purchase 
with an Amount in INR 1500 at NAV in INR 200 and Price in INR 100. Number of units were 15.

Another transaction with Folio number 15489856 / 75 from madame ltd. 
ISIN number INF123q34gt04q on Value date 11/03/2021. The Transaction type was Redemption 
with an Amount in INR 2500 at NAV in INR 300 and Price in INR 150. Number of units were 20.
"""

# Define regex patterns for each keyword
patterns = {
    "Folio number": r"Folio number\s*([\d\s/]+)",
    "Company name": r"Company name\s*([\w\s.]+)",
    "ISIN number": r"ISIN number\s*(INF\w+)",
    "Value date": r"Value date\s*([\d/]+)",
    "Transaction type": r"Transaction type\s*([\w\s]+)",
    "Amount in INR": r"Amount in INR\s*([\d,\.]+)",
    "NAV in INR": r"NAV in INR\s*([\d,\.]+)",
    "Price in INR": r"Price in INR\s*([\d,\.]+)",
    "Number of units": r"Number of units\s*(?:were\s*)?([\d,\.]+)"
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    transactions = re.split(r'(?=Folio number)', text)
    
    for transaction in transactions:
        data = {}
        for key, pattern in patterns.items():
            result = re.search(pattern, transaction)
            data[key] = result.group(1).strip() if result else None
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,None,INF456y78hj90u,11/02/2021,was Purchase \nwith an Amount in INR 1500 at N...,1500,200,100.,15.
1,15489856 / 75,None,INF123q34gt04q,11/03/2021,was Redemption \nwith an Amount in INR 2500 at...,2500,300,150.,20.


In [34]:
import pandas as pd
import re

# Sample unstructured text data
text = """
Some random text before the actual data.
Order ID: INF123q34gt04q with Folio number 15380577 / 05 bought some stocks from madame ltd. 
ISIN number INF456y78hj90u on Value date 11/02/2021. The Transaction type was Purchase 
with an Amount in INR 1500 at NAV in INR 200 and Price in INR 100. Number of units were 15.

Some random text in between.
Another transaction with Folio number 15489856 / 75 from madame ltd. 
ISIN number INF123q34gt04q on Value date 11/03/2021. The Transaction type was Redemption 
with an Amount in INR 2500 at NAV in INR 300 and Price in INR 150. Number of units were 20.

Some random text after the actual data.
"""

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"Folio number\s*([\d\s/]+)",
    "Company name": r"Company name\s*([\w\s.]+)",
    "ISIN number": r"ISIN number\s*(INF\w+)",
    "Value date": r"Value date\s*([\d/]+)",
    "Transaction type": r"Transaction type\s*([\w\s]+)",
    "Amount in INR": r"Amount in INR\s*([\d,\.]+)",
    "NAV in INR": r"NAV in INR\s*([\d,\.]+)",
    "Price in INR": r"Price in INR\s*([\d,\.]+)",
    "Number of units": r"Number of units\s*(?:were\s*)?([\d,\.]+)"
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    # Split the text into potential blocks of data
    transactions = re.split(r'\n\s*\n', text.strip())
    
    for transaction in transactions:
        data = {}
        for key, pattern in patterns.items():
            result = re.search(pattern, transaction, re.IGNORECASE)
            data[key] = result.group(1).strip() if result else None
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


    Folio number Company name     ISIN number  Value date  \
0  15380577 / 05         None  INF456y78hj90u  11/02/2021   
1  15489856 / 75         None  INF123q34gt04q  11/03/2021   

                                    Transaction type Amount in INR NAV in INR  \
0  was Purchase \nwith an Amount in INR 1500 at N...          1500        200   
1  was Redemption \nwith an Amount in INR 2500 at...          2500        300   

  Price in INR Number of units  
0         100.             15.  
1         150.             20.  


In [36]:
import pandas as pd
import re

# Sample highly unstructured text data
text = """
Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.

Another transaction. INF654z98xw21o executed on 2021-11-03.
Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.

Some trailing text...
"""

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"Folio\s*(?:number)?\s*[:=]\s*([\d\s/]+)",
    "Company name": r"Company\s*(?:name)?\s*[:=]\s*([\w\s.]+)",
    "ISIN number": r"ISIN\s*(?:number)?\s*[:=]\s*(INF\w+)",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"(PURCHASE|REDEMPTION)",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=]\s*INR?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=]\s*INR?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=]\s*INR?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    # Split the text into potential blocks of data
    transactions = re.split(r'\n\s*\n', text.strip())
    
    for transaction in transactions:
        data = {}
        for key, pattern in patterns.items():
            result = re.search(pattern, transaction, re.IGNORECASE)
            if result:
                data[key] = result.group(1).strip()
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


    Folio number  Value date Transaction type Number of units  \
0  15380577 / 05  2021-11-02         PURCHASE              15   
1            NaN  2021-11-03       Redemption             NaN   

       Company name     ISIN number  
0               NaN             NaN  
1  MADAME LTD. ISIN  INF123q34gt04q  


In [52]:
import pandas as pd
import re

# Sample highly unstructured text data
text = """
Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.

Another transaction. INF654z98xw21o executed on 2021-11-03.
Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.

Some trailing text...
"""

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": [
        r"Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)"
    ],
    "Company name": [
        r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)",
        r"bought stocks from\s*([\w\s.]+)"
    ],
    "ISIN number": [
        r"ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)"
    ],
    "Value date": [
        r"\b(\d{4}-\d{2}-\d{2})\b"
    ],
    "Transaction type": [
        r"\b(PURCHASE|REDEMPTION)\b",
        r"\b(Purchase|Redemption)\b"
    ],
    "Amount in INR": [
        r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
        r"INR\s*([\d,\.]+)\s*Amount"
    ],
    "NAV in INR": [
        r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)"
    ],
    "Price in INR": [
        r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)"
    ],
    "Number of units": [
        r"(\d+)\s*(?:units|Units)"
    ]
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    # Split the text into potential blocks of data
    transactions = re.split(r'\n\s*\n', text.strip())
    
    for transaction in transactions:
        print(f"Processing transaction block: {transaction}")  # Debugging print
        data = {}
        for key, pattern_list in patterns.items():
            found_match = False
            for pattern in pattern_list:
                matches = re.findall(pattern, transaction, re.IGNORECASE)
                if matches:
                    data[key] = matches[-1].strip()  # Take the last match
                    print(f"Matched {key}: {data[key]} with pattern: {pattern}")  # Debugging print
                    found_match = True
                    break  # Break the loop once a match is found
            if not found_match:
                data[key] = None
                print(f"No match for {key} in transaction block with patterns {pattern_list}")  # Debugging print
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


Processing transaction block: Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
Matched Folio number: 15380577 / 05 with pattern: Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)
Matched Company name: Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE with pattern: bought stocks from\s*([\w\s.]+)
Matched ISIN number: INF456y78hj90u with pattern: ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)
Matched Value date: 2021-11-02 with pattern: \b(\d{4}-\d{2}-\d{2})\b
Matched Transaction type: PURCHASE with pattern: \b(PURCHASE|REDEMPTION)\b
Matched Amount in INR: 1500. with pattern: Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)
No match for NAV in INR in transaction block with patterns ['NAV\\s*(?:in\\s*INR\\s*)?[:=\\-]?\\s*(?:INR)?\\s*([\\d,\\.]+)']
Matched Price in INR: 100. with pattern: Price\s*(?:in\s*INR\s*)?[:=\

In [51]:
import pandas as pd
import re

# Sample highly unstructured text data
text = """
Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.

Another transaction. INF654z98xw21o executed on 2021-11-03.
Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.

Some trailing text...
"""

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)",
    "ISIN number": r"ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    # Split the text into potential blocks of data
    transactions = re.split(r'\n\s*\n', text.strip())
    
    for transaction in transactions:
        print(f"Processing transaction block: {transaction}")  # Debugging print
        data = {}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, transaction, re.IGNORECASE)
            if matches:
                # If matches contain tuples, select the first non-empty group
                if isinstance(matches[0], tuple):
                    for match in matches:
                        match_value = next((m for m in match if m), None)
                        if match_value:
                            data[key] = match_value.strip()
                            break
                else:
                    data[key] = matches[-1].strip()  # Take the last match
                print(f"Matched {key}: {data[key]} with pattern: {pattern}")  # Debugging print
            else:
                data[key] = None
                print(f"No match for {key} in transaction block with pattern {pattern}")  # Debugging print
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


Processing transaction block: Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
Matched Folio number: 15380577 / 05 with pattern: Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)
Matched Company name: Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE with pattern: Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)
Matched ISIN number: INF456y78hj90u with pattern: ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)
Matched Value date: 2021-11-02 with pattern: \b(\d{4}-\d{2}-\d{2})\b
Matched Transaction type: PURCHASE with pattern: \b(PURCHASE|REDEMPTION)\b
Matched Amount in INR: 1500. with pattern: Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)
No match for NAV in INR in transaction block with pattern NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)
Matched Price in INR: 100. with pattern

In [53]:
import pandas as pd
import re

# Sample highly unstructured text data
text = """
Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.

Another transaction. INF654z98xw21o executed on 2021-11-03.
Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.

Some trailing text...
"""

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": [
        r"Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)"
    ],
    "Company name": [
        r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)",
        r"bought stocks from\s*([\w\s.]+)"
    ],
    "ISIN number": [
        r"ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)"
    ],
    "Value date": [
        r"\b(\d{4}-\d{2}-\d{2})\b"
    ],
    "Transaction type": [
        r"\b(PURCHASE|REDEMPTION)\b"
    ],
    "Amount in INR": [
        r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
        r"INR\s*([\d,\.]+)\s*Amount"
    ],
    "NAV in INR": [
        r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)"
    ],
    "Price in INR": [
        r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)"
    ],
    "Number of units": [
        r"(\d+)\s*(?:units|Units)"
    ]
}

# Function to extract information using regex patterns
def extract_info(text, patterns):
    extracted_data = []
    # Split the text into potential blocks of data
    transactions = re.split(r'\n\s*\n', text.strip())
    
    for transaction in transactions:
        print(f"Processing transaction block: {transaction}")  # Debugging print
        data = {}
        for key, pattern_list in patterns.items():
            found_match = False
            for pattern in pattern_list:
                matches = re.findall(pattern, transaction, re.IGNORECASE)
                if matches:
                    data[key] = matches[-1].strip()  # Take the last match
                    print(f"Matched {key}: {data[key]}")  # Debugging print
                    found_match = True
                    break  # Break the loop once a match is found
            if not found_match:
                data[key] = None
                print(f"No match for {key} in transaction block with patterns {pattern_list}")  # Debugging print
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info(text, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
print(df)


Processing transaction block: Random intro text...
On 2021-11-02, INF123q34gt04q was executed. 
Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
Matched Folio number: 15380577 / 05
Matched Company name: Madame Ltd. ISIN INF456y78hj90u.
The type was PURCHASE
Matched ISIN number: INF456y78hj90u
Matched Value date: 2021-11-02
Matched Transaction type: PURCHASE
Matched Amount in INR: 1500.
No match for NAV in INR in transaction block with patterns ['NAV\\s*(?:in\\s*INR\\s*)?[:=\\-]?\\s*(?:INR)?\\s*([\\d,\\.]+)']
Matched Price in INR: 100.
Matched Number of units: 15
Processing transaction block: Another transaction. INF654z98xw21o executed on 2021-11-03.
Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
Matched Folio number: 15489856/75
Matched Company name: MADAME LTD. ISIN
Matched ISIN number: I

In [55]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)",
    "ISIN number": r"ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    extracted_data = []
    
    for text in texts:
        data = {}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # If matches contain tuples, select the first non-empty group
                if isinstance(matches[0], tuple):
                    for match in matches:
                        match_value = next((m for m in match if m), None)
                        if match_value:
                            data[key] = match_value.strip()
                            break
                else:
                    data[key] = matches[-1].strip()  # Take the last match
            else:
                data[key] = None
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd. ISIN INF456y78hj90u.\n The type...,INF456y78hj90u,2021-11-02,PURCHASE,1500.,None,100.,15
1,15489856/75,MADAME LTD. ISIN,INF123q34gt04q,2021-11-03,Redemption,2500,300.,None,None


In [57]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)",
    "ISIN number": r"ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    extracted_data = []
    
    for text in texts:
        data = {}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # If matches contain tuples, select the first non-empty group
                if isinstance(matches[0], tuple):
                    for match in matches:
                        match_value = next((m for m in match if m), None)
                        if match_value:
                            data[key] = match_value.strip()
                            break
                else:
                    data[key] = matches[-1].strip()  # Take the last match
            else:
                data[key] = None
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd. ISIN INF456y78hj90u.\n The type...,INF456y78hj90u,2021-11-02,PURCHASE,1500.,None,100.,15
1,15489856/75,MADAME LTD. ISIN,INF123q34gt04q,2021-11-03,Redemption,2500,300.,None,None


In [59]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"Folio\s*(?:number|num)?\s*[:=\-]?\s*([\d\s/]+)",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)",
    "ISIN number": r"ISIN\s*(?:number)?\s*[:=\-]?\s*(INF\w+)",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    extracted_data = []
    
    for text in texts:
        data = {key: None for key in patterns.keys()}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # If matches contain tuples, select the first non-empty group
                if isinstance(matches[0], tuple):
                    for match in matches:
                        match_value = next((m for m in match if m), None)
                        if match_value:
                            data[key] = match_value.strip()
                            break
                else:
                    data[key] = matches[-1].strip()  # Take the last match
        if any(data.values()):  # Only add if there is at least one piece of data extracted
            extracted_data.append(data)
    return extracted_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd. ISIN INF456y78hj90u.\n The type...,INF456y78hj90u,2021-11-02,PURCHASE,1500.,None,100.,15
1,15489856/75,MADAME LTD. ISIN,INF123q34gt04q,2021-11-03,Redemption,2500,300.,None,None


In [65]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    """
    Transaction. INF654z98xw21o executed on 2021-11-05.
    Folio: 12345678 / 90 - Company: ABC Pvt. Ltd. ISIN: INF654z98xw21o.
    PURCHASE: Amount=3000 INR; NAV INR 250. Price was 120 INR. Units: 10.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"\b(\d{8} / \d{2})\b",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)",
    "ISIN number": r"\b(INF\w+)\b",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    all_data = []
    
    for text in texts:
        matches_dict = {key: [] for key in patterns.keys()}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    match_value = next((m for m in match if m), None)
                else:
                    match_value = match
                if match_value:
                    matches_dict[key].append(match_value.strip())
        
        # Ensure all lists in matches_dict are of the same length
        max_len = max(len(matches_dict[key]) for key in matches_dict.keys())
        for key in matches_dict:
            matches_dict[key].extend([None] * (max_len - len(matches_dict[key])))
        
        # Add the aligned matches to the all_data list
        for i in range(max_len):
            row = {key: matches_dict[key][i] for key in patterns.keys()}
            all_data.append(row)

    return all_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
df #&#8203;:citation[oaicite:0]{index=0}&#8203;


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd. ISIN INF456y78hj90u.\n The type...,INF123q34gt04q,2021-11-02,PURCHASE,1500.,None,100.,15
1,None,None,INF456y78hj90u,None,None,None,None,None,None
2,None,MADAME LTD. ISIN,INF654z98xw21o,2021-11-03,Redemption,2500,300.,None,None
3,None,None,INF123q34gt04q,None,None,None,None,None,None
4,12345678 / 90,ABC Pvt. Ltd. ISIN,INF654z98xw21o,2021-11-05,PURCHASE,3000,250.,None,None
5,None,None,INF654z98xw21o,None,None,None,None,None,None


In [68]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    """
    Transaction. INF654z98xw21o executed on 2021-11-05.
    Folio: 12345678 / 90 - Company: ABC Pvt. Ltd. ISIN: INF654z98xw21o.
    PURCHASE: Amount=3000 INR; NAV INR 250. Price was 120 INR. Units: 10.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"\b(\d{8} / \d{2}|\d{8}/\d{2})\b",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+)|bought stocks from\s*([\w\s.]+)",
    "ISIN number": r"\b(INF\w+)\b",
    "Value date": r"\b(\d{4}-\d{2}-\d{2})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    all_data = []
    
    for text in texts:
        matches_dict = {key: [] for key in patterns.keys()}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    match_value = next((m for m in match if m), None)
                else:
                    match_value = match
                if match_value:
                    matches_dict[key].append(match_value.strip())
        
        # Ensure all lists in matches_dict are of the same length
        max_len = max(len(matches_dict[key]) for key in matches_dict.keys())
        for key in matches_dict:
            matches_dict[key].extend([None] * (max_len - len(matches_dict[key])))
        
        # Add the aligned matches to the all_data list
        for i in range(max_len):
            row = {key: matches_dict[key][i] for key in patterns.keys()}
            all_data.append(row)

    return all_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd. ISIN INF456y78hj90u.\n The type...,INF123q34gt04q,2021-11-02,PURCHASE,1500.,None,100.,15
1,None,None,INF456y78hj90u,None,None,None,None,None,None
2,15489856/75,MADAME LTD. ISIN,INF654z98xw21o,2021-11-03,Redemption,2500,300.,None,None
3,None,None,INF123q34gt04q,None,None,None,None,None,None
4,12345678 / 90,ABC Pvt. Ltd. ISIN,INF654z98xw21o,2021-11-05,PURCHASE,3000,250.,None,None
5,None,None,INF654z98xw21o,None,None,None,None,None,None


In [70]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    """
    Transaction. INF654z98xw21o executed on 2021-11-05.
    Folio: 12345678 / 90 - Company: ABC Pvt. Ltd. ISIN: INF654z98xw21o.
    PURCHASE: Amount=3000 INR; NAV INR 250. Price was 120 INR. Units: 10.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"\b(\d{8} / \d{2}|\d{8}/\d{2})\b",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.))|bought stocks from\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.))",
    "ISIN number": r"\b(INF\w+)\b",
    "Value date": r"\b(\d{4}-\d{2}-\d{2}|\d{2} / \d{2} / \d{4})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    all_data = []
    
    for text in texts:
        matches_dict = {key: [] for key in patterns.keys()}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    match_value = next((m for m in match if m), None)
                else:
                    match_value = match
                if match_value:
                    matches_dict[key].append(match_value.strip())
        
        # Ensure all lists in matches_dict are of the same length
        max_len = max(len(matches_dict[key]) for key in matches_dict.keys())
        for key in matches_dict:
            matches_dict[key].extend([None] * (max_len - len(matches_dict[key])))
        
        # Add the aligned matches to the all_data list
        for i in range(max_len):
            row = {key: matches_dict[key][i] for key in patterns.keys()}
            all_data.append(row)

    return all_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd.,INF123q34gt04q,2021-11-02,PURCHASE,1500.,None,100.,15
1,None,None,INF456y78hj90u,None,None,None,None,None,None
2,15489856/75,MADAME LTD.,INF654z98xw21o,2021-11-03,Redemption,2500,300.,None,None
3,None,None,INF123q34gt04q,None,None,None,None,None,None
4,12345678 / 90,ABC Pvt. Ltd.,INF654z98xw21o,2021-11-05,PURCHASE,3000,250.,None,None
5,None,None,INF654z98xw21o,None,None,None,None,None,None


In [72]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    """
    Transaction. INF654z98xw21o executed on 2021/11/05.
    Folio: 12345678 / 90 - Company: ABC Pvt. Ltd. ISIN: INF654z98xw21o.
    PURCHASE: Amount=3000 INR; NAV INR 250. Price was 120 INR. Units: 10.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"\b(\d{8} / \d{2}|\d{8}/\d{2})\b",
    "Company name": r"Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.))|bought stocks from\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.))",
    "ISIN number": r"\b(INF\w+)\b",
    "Value date": r"\b(\d{4}[-/]\d{2}[-/]\d{2}|\d{2}[-/]\d{2}[-/]\d{4})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    all_data = []
    
    for text in texts:
        matches_dict = {key: [] for key in patterns.keys()}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    match_value = next((m for m in match if m), None)
                else:
                    match_value = match
                if match_value:
                    matches_dict[key].append(match_value.strip())
        
        # Ensure all lists in matches_dict are of the same length
        max_len = max(len(matches_dict[key]) for key in matches_dict.keys())
        for key in matches_dict:
            matches_dict[key].extend([None] * (max_len - len(matches_dict[key])))
        
        # Add the aligned matches to the all_data list
        for i in range(max_len):
            row = {key: matches_dict[key][i] for key in patterns.keys()}
            all_data.append(row)

    return all_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd.,INF123q34gt04q,2021-11-02,PURCHASE,1500.,None,100.,15
1,None,None,INF456y78hj90u,None,None,None,None,None,None
2,15489856/75,MADAME LTD.,INF654z98xw21o,2021-11-03,Redemption,2500,300.,None,None
3,None,None,INF123q34gt04q,None,None,None,None,None,None
4,12345678 / 90,ABC Pvt. Ltd.,INF654z98xw21o,2021/11/05,PURCHASE,3000,250.,None,None
5,None,None,INF654z98xw21o,None,None,None,None,None,None


In [74]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    """
    Transaction. INF654z98xw21o executed on 2021-11-05.
    Folio: 12345678 / 90 - Company: ABC Pvt. Ltd. ISIN: INF654z98xw21o.
    PURCHASE: Amount=3000 INR; NAV INR 250. Price was 120 INR. Units: 10.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"\b(\d{8} / \d{2}|\d{8}/\d{2})\b",
    "Company name": r"(?:Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.))|bought stocks from\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.)))",
    "ISIN number": r"\b(INF\w+)\b",
    "Value date": r"\b(\d{4}[-/]\d{2}[-/]\d{2}|\d{2}[-/]\d{2}[-/]\d{4})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    all_data = {key: [] for key in patterns.keys()}
    
    for text in texts:
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # Flatten tuples and clean up
                cleaned_matches = [m if isinstance(m, str) else next((x for x in m if x), '') for m in matches]
                all_data[key].extend(cleaned_matches)
    
    # Find the maximum length of the lists to align them
    max_len = max(len(lst) for lst in all_data.values())
    
    # Ensure all lists in all_data are of the same length
    for key in all_data:
        all_data[key].extend([None] * (max_len - len(all_data[key])))
    
    # Convert to list of dictionaries for DataFrame
    aligned_data = [{key: all_data[key][i] for key in all_data} for i in range(max_len)]
    
    return aligned_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Save to DataFrame
df = pd.DataFrame(extracted_info)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd.,INF123q34gt04q,2021-11-02,PURCHASE,1500.,300.,100.,15
1,15489856/75,MADAME LTD.,INF456y78hj90u,2021-11-03,Redemption,2500,250.,None,None
2,12345678 / 90,ABC Pvt. Ltd.,INF654z98xw21o,2021-11-05,PURCHASE,3000,None,None,None
3,None,None,INF123q34gt04q,None,None,None,None,None,None
4,None,None,INF654z98xw21o,None,None,None,None,None,None
5,None,None,INF654z98xw21o,None,None,None,None,None,None


In [75]:
import pandas as pd
import re

# Sample highly unstructured text data in a list
texts = [
    """
    On 2021-11-02, INF123q34gt04q was executed. 
    Folio: 15380577 / 05, bought stocks from Madame Ltd. ISIN INF456y78hj90u.
    The type was PURCHASE; Amount INR 1500. NAV INR: 200 and Price INR 100. 15 units.
    """,
    """
    Another transaction. INF654z98xw21o executed on 2021-11-03.
    Folio num 15489856/75 - Company: MADAME LTD. ISIN: INF123q34gt04q.
    Redemption: Amount=2500 INR; NAV INR 300. Price was 150 INR. Units: 20.
    """,
    """
    Transaction. INF654z98xw21o executed on 2021-11-05.
    Folio: 12345678 / 90 - Company: ABC Pvt. Ltd. ISIN: INF654z98xw21o.
    PURCHASE: Amount=3000 INR; NAV INR 250. Price was 120 INR. Units: 10.
    """,
    "Some trailing text..."
]

# Define regex patterns for each keyword dynamically
patterns = {
    "Folio number": r"\b(\d{8} / \d{2}|\d{8}/\d{2})\b",
    "Company name": r"(?:Company\s*(?:name)?\s*[:=\-]?\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.))|bought stocks from\s*([\w\s.]+(?:Pvt\. Ltd\.|Ltd\.)))",
    "ISIN number": r"\b(INF\w+)\b",
    "Value date": r"\b(\d{4}[-/]\d{2}[-/]\d{2}|\d{2}[-/]\d{2}[-/]\d{4})\b",
    "Transaction type": r"\b(PURCHASE|REDEMPTION)\b",
    "Amount in INR": r"Amount\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "NAV in INR": r"NAV\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Price in INR": r"Price\s*(?:in\s*INR\s*)?[:=\-]?\s*(?:INR)?\s*([\d,\.]+)",
    "Number of units": r"(\d+)\s*(?:units|Units)"
}

# Function to extract information using regex patterns from a list of texts
def extract_info_from_list(texts, patterns):
    all_data = {key: [] for key in patterns.keys()}
    
    for text in texts:
        local_data = {key: [] for key in patterns.keys()}
        for key, pattern in patterns.items():
            matches = re.findall(pattern, text, re.IGNORECASE)
            if matches:
                # Flatten tuples and clean up
                cleaned_matches = [m if isinstance(m, str) else next((x for x in m if x), '') for m in matches]
                local_data[key].extend(cleaned_matches)
        
        # Find the maximum length of the local_data lists to align them
        max_len = max(len(lst) for lst in local_data.values())
        
        # Ensure all lists in local_data are of the same length
        for key in local_data:
            local_data[key].extend([None] * (max_len - len(local_data[key])))
        
        # Add the aligned matches to the all_data lists
        for i in range(max_len):
            for key in patterns.keys():
                all_data[key].append(local_data[key][i])
    
    return all_data

# Extract information
extracted_info = extract_info_from_list(texts, patterns)

# Convert to list of dictionaries for DataFrame
aligned_data = [{key: extracted_info[key][i] for key in extracted_info} for i in range(len(extracted_info['Folio number']))]

# Save to DataFrame
df = pd.DataFrame(aligned_data)

# Display the DataFrame
(df)


,Folio number,Company name,ISIN number,Value date,Transaction type,Amount in INR,NAV in INR,Price in INR,Number of units
0,15380577 / 05,Madame Ltd.,INF123q34gt04q,2021-11-02,PURCHASE,1500.,None,100.,15
1,None,None,INF456y78hj90u,None,None,None,None,None,None
2,15489856/75,MADAME LTD.,INF654z98xw21o,2021-11-03,Redemption,2500,300.,None,None
3,None,None,INF123q34gt04q,None,None,None,None,None,None
4,12345678 / 90,ABC Pvt. Ltd.,INF654z98xw21o,2021-11-05,PURCHASE,3000,250.,None,None
5,None,None,INF654z98xw21o,None,None,None,None,None,None
